# ANÁLISIS DE SINIESTRALIDAD

### CUNEF MUCD (2021/22)

- Aitor Larriona Rementería
- Diego Cendán Bedregal

## LIBRERÍAS

Librerías básicas

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import missingno as msno
import seaborn as sns
import plotly.express as px
from statistics import mode, multimode
import time
import sklearn
import warnings
import scikitplot as skplt
import statsmodels.api as sapi

pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 5000)

Librerías para codificar variables categóricas

In [2]:
import category_encoders as ce
from category_encoders.target_encoder import TargetEncoder

Librerías para oversampling

In [3]:
from imblearn.over_sampling import SMOTE

In [4]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

Librerías para separar en train y test

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

Importamos el pipeline y pickle (para guardar los modelos)

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
import pickle

In [7]:
import pyarrow.parquet as pq
import pyarrow as pa

Importamos también las funciones necesarias en este notebook del notebook de funciones 00.

In [8]:
import import_ipynb

In [9]:
%run FUNCIONES

## Lectura y modificación de los datos

Lectura de la tabla .parquet que hemos creado en el notebook *01_EDA*

**Importante:** Antes de utilizar el formato parquet, se ha utilizado también la opción de escribir un csv. Sin embargo, esta opción distorsionaba los datos y no correspondían las clases del data frame pd_data en *01_EDA* con las clases del data frame que cargábamos en este notebook. Por ello, se ha optado por el formato parquet

In [10]:
table = pq.read_table('/Users/aitor/Desktop/Máster Ciencia de Datos/Aprendizaje automático/Machine-Learning/big_practice_data/pd_data.parquet')

In [11]:
df_pd_data = table.to_pandas()
df_pd_data

,C_YEAR,cos_C_MNTH,sin_C_MNTH,cos_C_WDAY,sin_C_WDAY,cos_C_HOUR,sin_C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,C_TRAF,Random
0,1999,0.866025,0.5,0.62349,0.781831,0.682553,-0.730836,2,2,03,None,01,02,02,01,98
1,1999,0.866025,0.5,0.62349,0.781831,0.682553,-0.730836,2,2,03,None,01,02,02,01,61
2,1999,0.866025,0.5,0.62349,0.781831,-0.57668,0.81697,2,1,01,None,02,02,02,03,22
3,1999,0.866025,0.5,0.62349,0.781831,-0.068242,-0.997669,2,3,QQ,QQ,01,02,01,01,75
4,1999,0.866025,0.5,0.62349,0.781831,-0.068242,-0.997669,2,3,QQ,QQ,01,02,01,01,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3902109,2014,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2,<NA>,None,01,None,None,None,None,36
3902110,2014,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2,<NA>,None,01,None,None,None,None,68
3902111,2014,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2,<NA>,None,01,None,None,None,None,30
3902112,2014,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2,<NA>,None,01,None,None,None,None,52


Comprobemos ahora que la clase de las variables es correcto y que no ha habido modificaciones de ningún tipo

In [12]:
df_pd_data.dtypes

C_YEAR          int64
cos_C_MNTH    Float64
sin_C_MNTH    Float64
cos_C_WDAY    Float64
sin_C_WDAY    Float64
cos_C_HOUR    Float64
sin_C_HOUR    Float64
C_SEV           int64
C_VEHS          Int64
C_CONF         object
C_RCFG         object
C_WTHR         object
C_RSUR         object
C_RALN         object
C_TRAF         object
Random          int64
dtype: object

Nuestro objetivo es predecir si habrá o no muertos en un accidente de tráfico. Es por ello mismo que nos quedaremos con todos los datos relacionados con las colisiones. En otras palabras,

In [13]:
valores_unicos(df_pd_data)

{'C_YEAR': [1999,
  2000,
  2001,
  2002,
  2003,
  2004,
  2005,
  2006,
  2007,
  2008,
  2009,
  2010,
  2011,
  2012,
  2013,
  2014],
 'cos_C_MNTH': [0.8660254037844387,
  0.5000000000000001,
  6.123233995736766e-17,
  -0.4999999999999998,
  -0.8660254037844387,
  -1.0,
  -0.8660254037844388,
  -0.5000000000000004,
  -1.8369701987210297e-16,
  0.5,
  0.8660254037844384,
  1.0,
  <NA>],
 'sin_C_MNTH': [0.49999999999999994,
  0.8660254037844386,
  1.0,
  0.8660254037844388,
  1.2246467991473532e-16,
  -0.4999999999999998,
  -0.8660254037844384,
  -1.0,
  -0.8660254037844386,
  -0.5000000000000004,
  -2.4492935982947064e-16,
  <NA>],
 'cos_C_WDAY': [0.6234898018587336,
  -0.22252093395631434,
  -0.900968867902419,
  -0.9009688679024191,
  -0.2225209339563146,
  0.6234898018587334,
  1.0,
  <NA>],
 'sin_C_WDAY': [0.7818314824680297,
  0.9749279121818236,
  0.43388373911755823,
  -0.433883739117558,
  -0.9749279121818235,
  -0.7818314824680299,
  -2.4492935982947064e-16,
  <NA>],
 'cos

---

## Separación en train y test

Eliminamos la variable objetivo C_SEV y creamos el data frame X. Por otro lado, con la variable C_SEV únicamente, creamos el objeto Y que representará nuestra variable target

In [14]:
X = df_pd_data.drop(["C_SEV"], axis = 1)
Y = df_pd_data.C_SEV

Mostramos el data frame X

In [15]:
X

,C_YEAR,cos_C_MNTH,sin_C_MNTH,cos_C_WDAY,sin_C_WDAY,cos_C_HOUR,sin_C_HOUR,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,C_TRAF,Random
0,1999,0.866025,0.5,0.62349,0.781831,0.682553,-0.730836,2,03,None,01,02,02,01,98
1,1999,0.866025,0.5,0.62349,0.781831,0.682553,-0.730836,2,03,None,01,02,02,01,61
2,1999,0.866025,0.5,0.62349,0.781831,-0.57668,0.81697,1,01,None,02,02,02,03,22
3,1999,0.866025,0.5,0.62349,0.781831,-0.068242,-0.997669,3,QQ,QQ,01,02,01,01,75
4,1999,0.866025,0.5,0.62349,0.781831,-0.068242,-0.997669,3,QQ,QQ,01,02,01,01,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3902109,2014,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,01,None,None,None,None,36
3902110,2014,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,01,None,None,None,None,68
3902111,2014,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,01,None,None,None,None,30
3902112,2014,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,01,None,None,None,None,52


In [16]:
Y = LabelEncoder().fit_transform(Y)
Y

array([1, 1, 1, ..., 1, 1, 1])

LabelEconder() otorga etiquetas categóricas según el valor que puede tomar nuestra variable target. En este caso, 2=1 (1 = No fallecidos) y 1=0 (0= Al menos un fallecido)

Separamos en train y test con una semilla de 0 y un tamaño de 75% y 25% para el conjunto de train y test respectivamente

In [17]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.25, random_state=0)

In [18]:
xtrain.dtypes

C_YEAR          int64
cos_C_MNTH    Float64
sin_C_MNTH    Float64
cos_C_WDAY    Float64
sin_C_WDAY    Float64
cos_C_HOUR    Float64
sin_C_HOUR    Float64
C_VEHS          Int64
C_CONF         object
C_RCFG         object
C_WTHR         object
C_RSUR         object
C_RALN         object
C_TRAF         object
Random          int64
dtype: object

---

## Preparación de los datos

Dado que tenemos un dataset desbalanceado (véase notebook *01_EDA*) nuestro objetivo será hacer uso de la técnica de oversampling, la cual explicaremos más adelante, para dotar de mayor información a los modelos a la hora de entrenar. Pero, para poder realizar el oversampling sobre el conjunto de datos, deberemos realizar varios cambios a las columnas, en función si son numéricas o de tipo object.
- Si las columnas son numéricas, se deberá imputar sus valores y missing y, posteriormente hacer un escalado.
- Si las columnas son categóricas, o de tipo object, se deberá hacer una codificación de las mismas.

Para realizar estas modificaciones haremos uso de los *Pipeline*. 

**Def.** Pipeline es un objeto que sirve para realizar una secuencia de diferentes transformaciones. 

Nosotros, en este caso, utilizaremos el pipeline para realizar, para cada tipo de columna, los pasos mencionados anteriormente. Es por ello que trataremos con dos pipelines, por un lado el de las columnas numéricas y, por otro, el de las columnas categóricas. 

Para las columnas numéricas realizaremos en primer lugar una imputación de los valores missing por la moda y después escalaremos las columnas mediante el StandardScaler (véase explicación de StandardScaler en *01_EDA*). Para las columnas categóricas, sin embargo, nos bastará únicamente con codificar la variable con el OneHotEncoding.

**Def.** El one hot encoding crea una columna nueva para cada valor distinto que exista en la variable a codificar. Cada registro se marca con el valor uno en la columna a la que pertenece el valor y los demás registros tomarán el valor 0.

In [19]:
scaled_transform = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                    ('scaler', StandardScaler())])

In [20]:
impute_NA = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent'))])

In [21]:
categorical_transform = Pipeline(steps=[('ohe', OneHotEncoder())])

Ahora, necesitamos seleccionar las columnas numéricas y categóricas. Esto lo hacemos de la siguiente forma

In [22]:
numeric_data = df_pd_data.select_dtypes(include=['int', "float"]).drop(["C_SEV"], axis=1).columns
int_data = df_pd_data.select_dtypes(include=['int']).drop(["C_SEV"], axis=1).columns
cat_data = df_pd_data.select_dtypes(include=['object']).columns

Para poder llevar a cabo el pipeline necesitamos del estimador *ColumnTransformer*. Esta función permite transformar, de forma separada, diferentes columnas y luego se concatenan los resultados para obtener un único dataset. Para nosotros es perfecto ya que no todas nuestras columnas del dataset necesitan ser modificadas de la misma manera como hemos podido ver anteriormente.

A las columnas numéricas les aplicaremos el pipeline numérico y a las categóricas el pipeline categórico

In [23]:
preprocessor = ColumnTransformer(
    transformers=[
        ('NA', impute_NA, numeric_data),
        ('cat', categorical_transform, cat_data),
        ("num", scaled_transform, int_data)])

Debido a que, de momento, no nos interesa escalar el test, hemos tenido problemas a la hora de intentar hacer un nuevo pipeline para el test. La única modificación que haremos en el test será la imputación de los NAs. Esta la haremos a mano de la siguiente forma:

In [24]:
preprocessor.fit(xtrain, ytrain)

ColumnTransformer(transformers=[('NA',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 Index(['C_YEAR', 'cos_C_MNTH', 'sin_C_MNTH', 'cos_C_WDAY', 'sin_C_WDAY',
       'cos_C_HOUR', 'sin_C_HOUR', 'C_VEHS', 'Random'],
      dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('ohe', OneHotEncoder())]),
                                 Index(['C_CONF', 'C_RCFG', 'C_WTHR', 'C_RSUR', 'C_RALN', 'C_TRAF'], dtype='object')),
                                ('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('scaler', StandardScaler())]),
                                 Index(['C_YEAR', 'C_VEHS', 'Random'], dtype='object'))])

Entrenamos nuestro estimador con los datos de training

Ahora ya podemos transformar nuestros datos mediante el atributo .transform. Lo haremos de los datos de training únicamente, de momento.

In [25]:
xtrain = preprocessor.transform(xtrain)

In [26]:
xtest = preprocessor.transform(xtest)

Para obtener los datos en formato de data frame necesitamos pasarlos, precisamente, a data frame

In [27]:
x_train = pd.DataFrame(xtrain, columns=get_feature_names(preprocessor))
x_test = pd.DataFrame(xtest, columns=get_feature_names(preprocessor))

/Users/aitor/Desktop/Máster Ciencia de Datos/Aprendizaje automático/Machine-Learning/big_practice/notebooks/FUNCIONES.py:334: UserWarning: Transformer imputer (type SimpleImputer) does not provide get_feature_names. Will return input column names if available
  warnings.warn("Transformer %s (type %s) does not "
/Users/aitor/opt/anaconda3/envs/ML_big_practice/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/aitor/Desktop/Máster Ciencia de Datos/Aprendizaje automático/Machine-Learning/big_practice/notebooks/FUNCIONES.py:334: UserWarning: Transformer scaler (type StandardScaler) does not provide get_feature_names. Will return input column names if available
  warnings.warn("Transformer %s (type %s) does not "


Comprobamos que, efectivamente, ya no tenemos ningún valor nulo en el dataset de training

In [28]:
xtrain

array([[ 2.00400000e+03, -1.00000000e+00,  1.22464680e-16, ...,
        -4.38597706e-01, -8.42755501e-01, -1.55427345e-01],
       [ 2.00300000e+03, -1.83697020e-16, -1.00000000e+00, ...,
        -6.57492328e-01, -7.37843983e-02,  4.33463624e-01],
       [ 2.00700000e+03, -5.00000000e-01,  8.66025404e-01, ...,
         2.18086160e-01, -7.37843983e-02,  1.54196427e+00],
       ...,
       [ 2.00700000e+03, -5.00000000e-01,  8.66025404e-01, ...,
         2.18086160e-01, -7.37843983e-02, -9.86802830e-01],
       [ 2.00400000e+03, -1.00000000e+00,  1.22464680e-16, ...,
        -4.38597706e-01,  1.46415781e+00, -6.75037023e-01],
       [ 2.00000000e+03,  5.00000000e-01,  8.66025404e-01, ...,
        -1.31417619e+00, -7.37843983e-02,  1.54196427e+00]])

In [29]:
valores_unicos(x_train, k=150)

{'NA_C_YEAR': [2004.0,
  2003.0,
  2007.0,
  2000.0,
  1999.0,
  2005.0,
  2013.0,
  2012.0,
  2001.0,
  2011.0,
  2008.0,
  2002.0,
  2014.0,
  2006.0,
  2010.0,
  2009.0],
 'NA_cos_C_MNTH': [-1.0,
  -1.8369701987210297e-16,
  -0.4999999999999998,
  0.5,
  -0.8660254037844387,
  -0.8660254037844388,
  0.8660254037844387,
  6.123233995736766e-17,
  -0.5000000000000004,
  0.8660254037844384,
  1.0,
  0.5000000000000001],
 'NA_sin_C_MNTH': [1.2246467991473532e-16,
  -1.0,
  0.8660254037844388,
  -0.8660254037844386,
  0.49999999999999994,
  -0.4999999999999998,
  1.0,
  -0.8660254037844384,
  -0.5000000000000004,
  -2.4492935982947064e-16,
  0.8660254037844386],
 'NA_cos_C_WDAY': [0.6234898018587334,
  -0.2225209339563146,
  0.6234898018587336,
  1.0,
  -0.900968867902419,
  -0.22252093395631434,
  -0.9009688679024191],
 'NA_sin_C_WDAY': [-0.7818314824680299,
  -0.9749279121818235,
  0.7818314824680297,
  -2.4492935982947064e-16,
  0.43388373911755823,
  0.9749279121818236,
  -0.43388373

In [30]:
x_train=x_train.drop(["NA_C_VEHS", "NA_C_YEAR", "NA_Random"], axis=1)
x_test=x_test.drop(["NA_C_VEHS", "NA_C_YEAR", "NA_Random"], axis=1)

In [31]:
columnas=x_train.drop(['NA_cos_C_MNTH', 'NA_sin_C_MNTH', 'NA_cos_C_WDAY', 'NA_sin_C_WDAY', 'NA_cos_C_HOUR', 'NA_sin_C_HOUR', "num_C_YEAR", "num_C_VEHS", "num_Random"], axis=1).columns

In [32]:
x_train[columnas] = x_train[columnas].astype("int")
x_test[columnas]=x_test[columnas].astype("int")

In [33]:
x_test.dtypes

NA_cos_C_MNTH    float64
NA_sin_C_MNTH    float64
NA_cos_C_WDAY    float64
NA_sin_C_WDAY    float64
NA_cos_C_HOUR    float64
NA_sin_C_HOUR    float64
ohe_x0_01          int64
ohe_x0_02          int64
ohe_x0_03          int64
ohe_x0_04          int64
ohe_x0_QQ          int64
ohe_x0_None        int64
ohe_x1_01          int64
ohe_x1_02          int64
ohe_x1_QQ          int64
ohe_x1_None        int64
ohe_x2_01          int64
ohe_x2_02          int64
ohe_x2_Q           int64
ohe_x2_None        int64
ohe_x3_01          int64
ohe_x3_02          int64
ohe_x3_Q           int64
ohe_x3_None        int64
ohe_x4_01          int64
ohe_x4_02          int64
ohe_x4_Q           int64
ohe_x4_None        int64
ohe_x5_01          int64
ohe_x5_03          int64
ohe_x5_QQ          int64
ohe_x5_None        int64
num_C_YEAR       float64
num_C_VEHS       float64
num_Random       float64
dtype: object

In [34]:
x_train.isna().any()

NA_cos_C_MNTH    False
NA_sin_C_MNTH    False
NA_cos_C_WDAY    False
NA_sin_C_WDAY    False
NA_cos_C_HOUR    False
NA_sin_C_HOUR    False
ohe_x0_01        False
ohe_x0_02        False
ohe_x0_03        False
ohe_x0_04        False
ohe_x0_QQ        False
ohe_x0_None      False
ohe_x1_01        False
ohe_x1_02        False
ohe_x1_QQ        False
ohe_x1_None      False
ohe_x2_01        False
ohe_x2_02        False
ohe_x2_Q         False
ohe_x2_None      False
ohe_x3_01        False
ohe_x3_02        False
ohe_x3_Q         False
ohe_x3_None      False
ohe_x4_01        False
ohe_x4_02        False
ohe_x4_Q         False
ohe_x4_None      False
ohe_x5_01        False
ohe_x5_03        False
ohe_x5_QQ        False
ohe_x5_None      False
num_C_YEAR       False
num_C_VEHS       False
num_Random       False
dtype: bool

---

Dado que tenemos los datos transformados, ya estamos en condiciones de introducir nuestro conjunto de testing en un algoritmo de oversampling

## Oversampling (SMOTE algorithm)

Como hemos podido comprobar en el notebook *01_EDA*, el dataset con el que estamos trabajando está realmente desbalanceado. Nuestra variable objetivo tiene muchos menos fallecidos que no fallecidos. Debido a esto, si introducimos en el modelo estos datos, no será posible obtener un buen modelo, pues no le estaremos dotando de suficiente información para que pueda predecir de manera correcta cuando haya algún fallecido. 

Teniendo en cuenta esto, existen dos grandes técnicas para solucionar este problema. Estas son *undersampling* y *oversampling*. Esta última consiste en generar datos "aleatorios" en la clase mayoritaria, para así poder dotar de información suficiente al entrenamiento del modelo. En cambio, el *undersampling* consiste en lo contrario, eliminar datos "aleatorios" de la clase mayoritaria de manera que ambas clases queden menos desvalanceadas. 

Nosotros haremos uso de *oversampling* y para ello utilizaremos uno de los algoritmos más utilizados en estas técnicas (SMOTE).

**Def.** SMOTE (Synthetic Minority Oversampling Technique) utiliza un algoritmo de vecino k-más cercano para crear datos sintéticos. SMOTE primero comienza eligiendo datos aleatorios de la clase minoritaria, luego se establecen los k vecinos más cercanos de los datos. Los datos sintéticos se harían entonces entre los datos aleatorios y el vecino k más cercano seleccionado al azar.

**Ejemplo.** En el siguiente ejemplo podremos ver cómo trata de crear el algoritmo los nuevos datos sintéticos

<table>
  <tr>
     <td>Before SMOTE</td>
     <td>After SMOTE</td>
  </tr>
  <tr>
    <td><img src="SMOTE_before.png" width="300" hspace="100"/> 
    <td><img src="SMOTE_after.png" width="300"/>
  </tr>
 </table>

In [35]:
print("Antes del OverSampling, número de '1's: {}".format(sum(ytrain == 1)))
print("Antes del OverSampling, número de '0's: {} \n".format(sum(ytrain == 0)))

Antes del OverSampling, número de '1's: 2884022
Antes del OverSampling, número de '0's: 42563 



In [36]:
sm = SMOTE(random_state = 2, sampling_strategy=0.5)
X_train_oversampled, y_train_oversampled = sm.fit_resample(x_train, ytrain.ravel())

In [37]:
print('Después del OverSampling, el tamaño de train_X: {}'.format(X_train_oversampled.shape))
print('Después del OverSampling, el tamaño de train_y: {} \n'.format(y_train_oversampled.shape))
 
print("Después del OverSampling, número de '1's: {}".format(sum(y_train_oversampled == 1)))
print("Después del OverSampling, número de '0's: {}".format(sum(y_train_oversampled == 0)))

Después del OverSampling, el tamaño de train_X: (4326033, 35)
Después del OverSampling, el tamaño de train_y: (4326033,) 

Después del OverSampling, número de '1's: 2884022
Después del OverSampling, número de '0's: 1442011


Una vez tenemos los conjuntos de training y testing definitivos, vamos a pasarlos a formato parquet para poder utilizarlos en otros notebooks.

In [40]:
table_X_train = pa.Table.from_pandas(X_train_oversampled, preserve_index=False)

In [41]:
pq.write_table(table_X_train, '/Users/aitor/Desktop/Máster Ciencia de Datos/Aprendizaje automático/Machine-Learning/big practice_data/X_train_oversampled.parquet')

FileNotFoundError: [Errno 2] Failed to open local file '/Users/aitor/Desktop/Máster Ciencia de Datos/Aprendizaje automático/Machine-Learning/big practice_data/X_train_oversampled.parquet'. Detail: [errno 2] No such file or directory

In [ ]:
table_ytrain = pa.Table.from_pandas(pd.DataFrame(y_train_oversampled), preserve_index=False)

In [ ]:
pq.write_table(table_ytrain, '/Users/aitor/Desktop/Máster Ciencia de Datos/Aprendizaje automático/Machine-Learning/big practice_data/y_train_oversampled.parquet')

In [ ]:
table_xtest = pa.Table.from_pandas(x_test, preserve_index=False)

In [ ]:
pq.write_table(table_xtest, '/Users/aitor/Desktop/Máster Ciencia de Datos/Aprendizaje automático/Machine-Learning/big practice_data/x_test.parquet')

In [ ]:
table_ytest = pa.Table.from_pandas(pd.DataFrame(ytest), preserve_index=False)

In [ ]:
pq.write_table(table_ytest, '/Users/aitor/Desktop/Máster Ciencia de Datos/Aprendizaje automático/Machine-Learning/big practice_data/ytest.parquet')